In [8]:
import os
import sys

sys.path.append(os.getcwd())

import numpy as np


class KalmanFilter:
    def __init__(self, 
                 initial_x: float, 
                 initial_v: float,
                 accel_variance: float) -> None:
                
        # Mean of the state GRV (Gaussian Random Variable)
        self._x = np.array([initial_x, initial_v])
        self._accel_variance = accel_variance

        # Covariance of the state GRV (Gaussian Random Variable)
        self._P = np.eye(2)
    
    @property
    def pos(self) -> float:
        return self._x[0]
    
    @property
    def vel(self) -> float:
        return self._x[1]
    
    @property
    def cov(self) -> np.ndarray:
        return self._P

    @property
    def mean(self) -> np.ndarray:
        return self._x
    
    def predict(self, dt: float) -> None:
        # dt here is the time step
        # x = F x
        # P = F P F^T + Q (Q = G G^T * accel_variance)
        F = np.array([[1, dt], [0, 1]])
        G = np.array([0.5 * dt ** 2, dt]).reshape((2,1))
        
        new_x = F.dot(self._x)
        new_P = F.dot(self._P).dot(F.T) + G.dot(G.T) * self._accel_variance
        
        self._x = new_x
        self._P = new_P
    
    def update(self, mess_value: float, mess_variance: float):
        # y = z - H x
        # S = H P H^T + R
        # K = P H^T S^-1
        # x = x + K y
        # P = (I - K H) P
        
        H = np.array([1, 0]).reshape((1, 2))
        z = np.array([mess_value])
        R = np.array([mess_variance])
        
        y = z - H.dot(self._x)
        S = H.dot(self._P).dot(H.T) + R
        K = self._P.dot(H.T).dot(np.linalg.inv(S))
        
        new_x = self._x + K.dot(y)
        new_P = (np.eye(2) - K.dot(H)).dot(self._P)
        
        self._x = new_x
        self._P = new_P

In [10]:
kf = KalmanFilter(
    initial_x=0.2,
    initial_v=0.5,
    accel_variance=1.2
)

In [11]:
kf.predict(dt=0.1)

In [15]:
kf.pos

0.3